In [4]:
import os
import time

delete_remaining = True
for i in os.listdir('training'):
    if i.endswith('.yaml'):
        os.remove(os.path.join('training',i))

n_gpus = 4#6
cpus = 4
memory = "64Gi"

batch_size=4
#This names will only work if you change the "data_config.py" file and add these names and their respective paths.
data_names=[
    "Deforestation-256-NGB",  
    "Deforestation-512-NGB",
    "Deforestation-256-NDVI",
    "Deforestation-512-NDVI",
    "Deforestation-256-EVI",
    "Deforestation-512-EVI",
]
lrs= [ 
    0.001,
    #0.005,
    0.0001,
    #0.0005
    0.00001
]#,0.00001,0.00005] 
max_epochs=200
embed_dim=256

multi_pred_weights =  None#[0.8, 0.2, 0.2, 0.8, 1.0]

net_G='ChangeFormerV6'        #ChangeFormerV6 is the finalized verion

lr_policy='linear'            #Choices: linear,step
optimizers=[
    'adamw',
    'adam',
    'sgd'
]                #Choices: sgd (set lr to 0.01), adam, adamw
losses=[
    'miou',
    'ce'
       ]                        #Choices: ce, fl (Focal Loss), miou
multi_scale_train=True
multi_scale_infer=False
shuffle_AB=False
###################################################################################


In [5]:
N=0
LGPU = False
gpus=[str(i) for i in range(n_gpus)]
visdev = "CUDA_VISIBLE_DEVICES=" + ",".join(gpus) + " "
#visdev = ""
checkpoint_root='/dataset/area_89/ChangeFormer/new_processing3/checkpoints'#'/dataset/ChangeFormer/checkpoints'
vis_root='/dataset/area_89/ChangeFormer/new_processing3/vis'#'/dataset/ChangeFormer/vis'
pretrain = "/dataset/ChangeFormer/pretrained_changeformer.pt"
split='train'         
split_val='test'      
for lr in lrs:
    for loss in losses:
        for optimizer in optimizers:
            for data_name in data_names:

                img_size=int(data_name.split('-')[-2])
                if img_size==256:
                    n_gpus=1
                elif img_size==512:
                    n_gpus=4
                gpus=[str(i) for i in range(n_gpus)]
                visdev = "CUDA_VISIBLE_DEVICES=" + ",".join(gpus) + " "

                project_name=f'CD_{time.time()}_{net_G}_{data_name}_b{batch_size}_lr{lr}_{optimizer}_{split}_{split_val}_{max_epochs}_{lr_policy}_{loss}_multi_train_{multi_scale_train}_multi_infer_{multi_scale_infer}_shuffle_AB_{shuffle_AB}_embed_dim_{embed_dim}'

                cmd = f'{visdev}python3 /dataset/ChangeFormer/main_cd.py --img_size {img_size} --loss {loss} --checkpoint_root {checkpoint_root} --vis_root {vis_root} --lr_policy {lr_policy} --optimizer {optimizer} --split {split} --split_val {split_val} --net_G {net_G} --multi_scale_train {multi_scale_train} --multi_scale_infer {multi_scale_infer} --gpu_ids {",".join(gpus)} --max_epochs {max_epochs} --project_name {project_name} --batch_size {batch_size} --shuffle_AB {shuffle_AB} --data_name {data_name} --lr {lr} --embed_dim {embed_dim}'
                if pretrain is not None:
                    cmd += f' --pretrain {pretrain}'

                if multi_pred_weights is not None:
                    cmd += f' --multi_pred_weights {",".join(multi_pred_weights)}'

                file = f'''
apiVersion: batch/v1
kind: Job
metadata:
  name: job-run-train{N}
spec:
  template:
    spec:
      containers:
      - name: pod-process
        image: gitlab-registry.nrp-nautilus.io/msapwz/container_cf
        workingDir: /dataset
        command: ["/bin/sh","-c"]
        args:
        - "{cmd}"
        volumeMounts:
        - name: dataset
          mountPath: /dataset
        - name: dshm
          mountPath: /dev/shm
        resources:
          limits:
            memory: {memory}
            cpu: {cpus}
            nvidia.com/gpu: "{n_gpus}"
          requests:
            memory: {memory}
            cpu: {cpus}
            nvidia.com/gpu: "{n_gpus}" 
      volumes:
      - name: dataset
        persistentVolumeClaim:
            claimName: volume-deforestation-project-many
      - name: dshm
        emptyDir:
          medium: Memory
          sizeLimit: 64Gi
      restartPolicy: Never
        '''
                if LGPU:
                    file +='''
      #affinity:
      #  nodeAffinity:
      #    requiredDuringSchedulingIgnoredDuringExecution:
      #      nodeSelectorTerms:
      #        - matchExpressions:
      #            - key: nvidia.com/gpu.product
      #              operator: In
      #              values:
      #                - NVIDIA-RTX-A6000
      '''
                    
                file+='''
    backoffLimit: '''
                N+=1

                filename = f'./training/ChangeFormer_{project_name}_imgsize={img_size}.yaml'

                with open(filename,mode='w') as f:
                    f.write(file)


In [6]:
with open("training/run_all.sh",mode="w") as f:
    for i in os.listdir('training'):
        if i.endswith("yaml"):
            f.write(f"kubectl create -f {i}\n")
        
    